# Intro to PyCAMPS

The Centralized Automated Modeling of Power Systems library in Python (PyCAMPS) is a library that allows users to easily set up power systems, analyze stability, and run simulations. This notebook will go through a simple example and walk you through the different functions you can use.

### Boilerplate

In [3]:
!pip install pycamps==0.0.9
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: pycamps
    Found existing installation: PyCAMPS 0.0.8
    Uninstalling PyCAMPS-0.0.8:
      Successfully uninstalled PyCAMPS-0.0.8


In [4]:
from pycamps.modules import Type4_1, RLLoad, LongLine
from pycamps.simulation import Dynamics, PowerSystem, StateSpace
from pycamps.logger import configure_logging
from sympy import symbols, pprint
from sympy import latex, init_printing
init_printing(use_latex='mathjax')
from IPython.display import display, Math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
os.environ['LOG_LEVEL'] = 'INFO'
logger = configure_logging(log_to_console=True, log_to_file=False)

In [5]:
def display_equations(M):
  for i, equation in enumerate(M):
    latex_equations = latex(equation, mul_symbol='dot').replace('{eq}\'', '{{eq}^{\prime}}')
    latex_equations = latex_equations.replace('{xd}\'', '{{xd}^{\prime}}')
    latex_equations = latex_equations.replace('dphidt', '\\frac{d \phi}{dt}')
    latex_equations = r"\textbf{(" + str(i+1) + r")} \quad " + latex_equations
    display(Math(latex_equations))

def display_matrix(M):
  latex_equations = latex(M, mul_symbol='dot').replace('{eq}\'', '{{eq}^{\prime}}')
  latex_equations = latex_equations.replace('{xd}\'', '{{xd}^{\prime}}')
  latex_equations = latex_equations.replace('dphidt', '\\frac{d \phi}{dt}')
  display(Math(latex_equations))

In [6]:
import plotly.io as pio
pio.renderers.default = "colab"

import plotly.graph_objects as go

def plot_trajectory(time, states):
    fig = go.Figure()
    n = max(1, len(time) // 1000)

    # Create traces for each state
    for variable_name, array in states.items():
        fig.add_trace(go.Scatter(x=time[::n], y=array[::n], name=variable_name, visible=True))

    buttons = [ dict(
                    label='All',
                    method="update",
                    args=[{"visible": [True]*len(states)},
                          {"title": 'All'}]
                )
              ]
    # Create updatemenus (checkboxes)
    updatemenus = [
        dict(
            type="dropdown",
            direction="down",
            active=0,
            x=1.02,
            y=1,
            yanchor="top",
            buttons=buttons + [
                dict(
                    label=variable_name,
                    method="update",
                    args=[{"visible": [var == variable_name for var in states.keys()]},
                          {"title": variable_name}]
                ) for variable_name in states.keys()
            ],
        )
    ]

    # Update layout
    fig.update_layout(
        updatemenus=updatemenus,
        showlegend=True,
        legend=dict(x=1.02, y=0.5),
        margin=dict(r=200),  # Add right margin for the menu
        width=1000,  # Increase overall width
        height=600,  # Set a reasonable height
    )

    # Add a rangeslider
    fig.update_xaxes(rangeslider_visible=True)

    # Show the plot
    fig.show()

### Setting up the power system

We're going to go through a simple example of setting up a power system and running some simulations.

Let's initialize some components of our power system. For this example, our synchronous machine will be a Type 4-1 Generator connected to a standard RL load.

In [7]:
SM = Type4_1('G1', BaseSpeed=377)
TL = LongLine('TL_1_2', BaseSpeed=377)
L = RLLoad('L1', BaseSpeed=377)
Modules = [SM, TL, L]

Let's take a look at our components' state space equations to verify they're correct.

In [8]:
display_equations(SM.StateSpaceEquations)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
display_equations(TL.StateSpaceEquations)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
display_equations(L.StateSpaceEquations)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now let's connect our components together via buses and initialize a PowerSystem object.

In [11]:
Bus1 = [[SM], [TL,'L']]
Bus2 = [[L], [TL,'R']]
Buses = [Bus1, Bus2]

# Initialize power system
PS = PowerSystem("Example_1", Modules, Buses)

INFO - Produced G-matrix:
[[1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1.]]
INFO - The KCL constraints between modules are:
-------------------------
1.0*iInLd_TL_1_2 - 1.0*(RS_G1*eqprime_G1*cos(delta_G1) - RS_G1*vSd_G1 + eqprime_G1*xdprime_G1*sin(delta_G1) - vSq_G1*xdprime_G1)/(RS_G1**2 + xdprime_G1**2) = 0
1.0*iInLq_TL_1_2 - 1.0*(RS_G1*eqprime_G1*sin(delta_G1) - RS_G1*vSq_G1 - eqprime_G1*xdprime_G1*cos(delta_G1) + vSd_G1*xdprime_G1)/(RS_G1**2 + xdprime_G1**2) = 0
1.0*iInRd_TL_1_2 + 1.0*iLd_L1 = 0
1.0*iInRq_TL_1_2 + 1.0*iLq_L1 = 0
-------------------------
INFO - Produced state space for Example_1


Congratulations, you've made your first power system! Let's take a look at its state space.

In [12]:
display_equations(PS.StateSpaceEquations)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Examining the dynamics

Alright, we've set up our power system and verified the state space equations, so let's start solving for equilibrium! Let's instantiate a Dynamics object and find out which component parameters we need to plug in.

In [13]:
simulator = Dynamics(PS)
simulator.get_required_parameters()

{'G1': [J_G1, D_G1, xdprime_G1, RS_G1, eqprime_G1, Pm_G1],
 'TL_1_2': [RTL_TL_1_2, CTL_TL_1_2, LTL_TL_1_2],
 'L1': [RL_L1, LL_L1]}

For this example, we've provided you with the required parameters for our components.

In [14]:
params_dictionary = {'J_G1': 76531.33492530156,
                     'D_G1': 1,
                     'xdprime_G1': 0.005,
                     'RS_G1': 0.0001,
                     'eqprime_G1': -1.35,
                     'Pm_G1': 1.4786690000000817,
                     'RTL_TL_1_2': 0.02978336677167938,
                     'CTL_TL_1_2': 0.001,
                     'LTL_TL_1_2': 0.40681749584779936,
                     'RL_L1': 2.3125,
                     'LL_L1': 0.9588}

In [15]:
simulator.load_new_params(params_dictionary=params_dictionary)
xf = simulator.solve_equilibrium(method='hybr')

INFO - Solving for equilibrium values of Example_1
INFO - Equilibrium values:
-------------------------
delta_G1: 1.7240938868791578
omega_G1: 0.9999999763315144
vTLLd_TL_1_2: 0.20384754416483242
vTLLq_TL_1_2: -1.333223658211601
iTLMd_TL_1_2: -0.181576837552206
iTLMq_TL_1_2: -0.46280328107422397
vTLRd_TL_1_2: 0.020979052457981817
vTLRq_TL_1_2: -1.2455712151093845
iLd_L1: -0.1828224021007726
iLq_L1: -0.4628242287792925
-------------------------
INFO - Saved equilibrium values to results/equilibrium/Example_1_Solved.mat


Great! You should see a set of equilibrium values for the state variables of the power system. Let's now analyze the stability of our system using linearized analysis.

In [16]:
simulator.linearized_analysis(xf=xf)

INFO - Performing linearized analysis of Example_1
INFO - No unstable eigenvalues


(None, None)

Now we can simulate the trajectory of the power system over a set time period and see how the state changes over time.

In [17]:
time, states = simulator.simulate_trajectory(xf=xf, simulation_time=0.3, method='LSODA')
plot_trajectory(time, states)

INFO - Simulating trajectory of Example_1
INFO - Ending simulation at t = 0.3
INFO - Saved trajectory values to results/trajectory/Example_1_Simulation.mat


Try messing around with the starting values and seeing how the system reacts when xf is not at equilibrium.

Hint: Multiply xf by some constant value like 1.1 and run the previous cell again.